**Antes de ejecutar el codigo se debe de instalar la librería openpyxl en la terminal**

In [6]:
#Librerias
import oci
import ads
import logging
import numpy as np
import os
import pandas as pd
import shutil
import tempfile
import warnings
from ads.dataset.factory import DatasetFactory
from os import path
warnings.filterwarnings("ignore")
logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

In [7]:
#Autenticacion
signer = oci.auth.signers.get_resource_principals_signer()
config = {'region': signer.region, 'tenancy': signer.tenancy_id}
ai_language_client = oci.ai_language.AIServiceLanguageClient(config=config , signer=signer)

In [8]:
#Leer el excel que contiene los tweets
df=pd.read_excel("tweets_premier_league_footballersv3_esp.xlsx")

#Seleccionamos la columna de interes donde se encuentran los tweets
df_sentiment = df['tweet']

In [9]:
sentimientos_resultados = []
#Ciclo donde se extrae los sentimiendos de la columna de interes y se aplica el servicio de IA Language
for sentimiento in df_sentiment[0:518]:
    if isinstance(sentimiento, str):
        batch_detect_language_sentiments_response = ai_language_client.batch_detect_language_sentiments(
            batch_detect_language_sentiments_details=oci.ai_language.models.BatchDetectLanguageSentimentsDetails(
                documents=[
                    oci.ai_language.models.TextDocument(
                        key="0",
                        text=sentimiento,
                        language_code="es")],
                compartment_id=os.environ["NB_SESSION_COMPARTMENT_OCID"]),
            level=["SENTENCE"])

        if batch_detect_language_sentiments_response.data.documents and \
           batch_detect_language_sentiments_response.data.documents[0].sentences:

            # Obtener el sentimiento del tweet
            sentimiento_resultado = batch_detect_language_sentiments_response.data.documents[0].sentences[0].sentiment
        else:
            sentimiento_resultado = "El sentimiento no se pudo analizar."
    else:
        sentimiento_resultado = "Tweet vacio."

    sentimientos_resultados.append(sentimiento_resultado)
    
#Se agrega una columna con el resultado de los sentimientos
df['Sentimiento_OCI_Language'] = sentimientos_resultados

df.to_excel("resultado_analisis_sentimiento.xlsx", index=False)